### Imports

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

### Data Settings

In [ ]:
file_path = 'Desktoppanel.csv'
data = pd.read_csv(file_path)

data['before_2013'] = data['epq8001'] - data['epq8902'] - data['epq8905']
temporary_workers = data['epq8002'].fillna(0) + data['epq8003'].fillna(0) + data['epq8004'].combine_first(data['before_2013'])

y = temporary_workers
X = data[['aq1001', 'aq1002', 'aq2001', 'aq2003', 'aq3003', 'mq1001']]
# 다수사업장 여부, 본사 여부, 조직유형, 상장 여부, 설립년도, 노조여부 

model = sm.OLS(data).fit()

print(model.summary())

### Visualization

In [ ]:
plt.figure(figsize=(10, 6))
sns.residplot(x=model.fittedvalues, y=model.resid, lowess=True, line_kws={"color": "red"})
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()